In [1]:
import numpy as np
import math

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris

from poliastro.bodies import Sun,Venus, Earth, Jupiter, Saturn
from poliastro.threebody import flybys
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.iod import izzo
from poliastro.plotting import OrbitPlotter2D
from poliastro.util import norm
import poliastro.twobody.propagation as Propagation

from scipy import optimize as opt
 
solar_system_ephemeris.set("jpl")

<ScienceState solar_system_ephemeris: 'jpl'>

In [2]:
#tvectors [arrival after 2020, route transits in reverse]

In [3]:
route = [Saturn, Jupiter, Earth]

In [168]:
#calculator
def trajectory_calculator(t, route):
    
    if len(t) is not len(route):
        disp('len of t is not len of route')
        return
    
    sol=dict();
        
    t=[ti*u.year for ti in t];
    t_arrive = Time("2020-01-01", scale="utc")+t[0];
    
    
    #create flyby dates
    t_event=[t_arrive]
    for i in range(len(t)-1):
        t_event.append(t_event[i]-t[i+1])
        
    sol["events"] = t_event
    
    #create planet orbits
    o_planets=[];
    for i in range(len(route)):
        o_planets.append(Orbit.from_body_ephem(route[i],t_event[i]));
            
    sol["planet_orbits"] = o_planets
    

    
    #determine transfer velocities
    v_outs=[]
    v_ins =[]
    
    for i in range(len(route)-1):
        
        o_out = o_planets[i+1]
        o_in  = o_planets[i];
        
        (v_out, v_in), = izzo.lambert(Sun.k, o_out.r, o_in.r,t[i+1])
        
        v_outs.append(v_out);
        v_ins.append(v_in)
    
    #calculate transfer trajectories
    o_transfer=[]
    for i in range(len(route)-1):
        o_transfer.append(Orbit.from_vectors(Sun,o_planets[i+1].r, v_outs[i]))
    
    sol["transfer_orbits"] = o_transfer;
    
    #calculate deltav
    #insertion
    deltavs=[];
    deltavs.append(o_planets[0].v-v_ins[0])
    
    #flybys
    for i in range(len(route)-2):
        print(i)        
        deltavs.append(v_outs[i]-v_ins[i+1])
        
    #launch
    deltavs.append(v_outs[-1] - o_planets[-1].v)
    
    sol["deltavs"] = deltavs
    
    return sol
    

In [169]:
tguess = [10,10,5,0.3,0.3]
route = [Saturn, Jupiter,Earth, Venus,Earth]

In [170]:
solution=trajectory_calculator(tguess,route)

0
1
2


/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2029-12-31 12:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2020-01-01 00:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2014-12-31 18:01:09.184. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2014-09-13 04:13:09.182. Use Time(..., scale='tdb') instead.

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to 

In [171]:
solution["events"]

[<Time object: scale='utc' format='iso' value=2029-12-31 12:00:00.000>,
 <Time object: scale='utc' format='iso' value=2020-01-01 00:00:00.000>,
 <Time object: scale='utc' format='iso' value=2014-12-31 18:00:02.000>,
 <Time object: scale='utc' format='iso' value=2014-09-13 04:12:02.000>,
 <Time object: scale='utc' format='iso' value=2014-05-26 14:24:02.000>]

In [172]:
solution["planet_orbits"]

[9 x 10 AU x 22.6 deg (ICRS) orbit around Sun (☉) at epoch 2029-12-31 12:01:09.184 (TDB),
 5 x 5 AU x 23.2 deg (ICRS) orbit around Sun (☉) at epoch 2020-01-01 00:01:09.184 (TDB),
 1 x 1 AU x 23.4 deg (ICRS) orbit around Sun (☉) at epoch 2014-12-31 18:01:09.184 (TDB),
 1 x 1 AU x 24.4 deg (ICRS) orbit around Sun (☉) at epoch 2014-09-13 04:13:09.182 (TDB),
 1 x 1 AU x 23.4 deg (ICRS) orbit around Sun (☉) at epoch 2014-05-26 14:25:09.185 (TDB)]

In [173]:
solution["transfer_orbits"]

[4 x 10 AU x 23.2 deg (HCRS) orbit around Sun (☉) at epoch J2000.000 (TT),
 1 x 6 AU x 23.2 deg (HCRS) orbit around Sun (☉) at epoch J2000.000 (TT),
 0 x 1 AU x 23.1 deg (HCRS) orbit around Sun (☉) at epoch J2000.000 (TT),
 0 x 1 AU x 24.9 deg (HCRS) orbit around Sun (☉) at epoch J2000.000 (TT)]

In [188]:
op = OrbitPlotter2D();

op.plot(solution["planet_orbits"][4],label=route[4])
op.plot(solution["planet_orbits"][3],label=route[3])
op.plot(solution["planet_orbits"][2],label=route[2])
op.plot(solution["planet_orbits"][1],label=route[1])
op.plot(solution["planet_orbits"][0],label=route[0])


op.plot(solution["transfer_orbits"][3],label=[route[3],route[4]])
op.plot(solution["transfer_orbits"][2],label=[route[2],route[3]])
op.plot(solution["transfer_orbits"][1],label=[route[1],route[2]])
op.plot(solution["transfer_orbits"][0],label=[route[0],route[1]])

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/propagation.py:232: UserWarning:

Frame <class 'astropy.coordinates.builtin_frames.icrs.ICRS'> does not support 'obstime', time values were not returned



FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [185]:
norm(dv[0].to(u.km/u.s)).value

2.580704246331175

In [189]:
dv=solution['deltavs']

In [190]:
dv

[<Quantity [ -63986.72176667, 200688.79797855,  73120.42293775] km / d>,
 <Quantity [ 7.6703037 ,-8.8104913 ,-4.41186038] km / s>,
 <Quantity [-21.2416019 ,  0.6727219 , -0.17814002] km / s>,
 <Quantity [ 34.79494405,-14.93147861, -7.58601605] km / s>,
 <Quantity [ -1.75945522, 12.05566736,  3.85822842] km / s>]

In [191]:
dvmag = [norm(v.to(u.km/u.s)).value for v in dv]

In [192]:
dvmag

[2.580704246331175,
 12.486906253489856,
 21.25299837302219,
 38.6158623362765,
 12.779699709594443]